# Работа с Excel

v 0.1

Датасет: себестоимостьА_в1.xlsx

In [1]:
import xlwings as xw
import numpy as np
import pandas as pd
from xlwings import constants 

## 1. Используя xlwings:

In [2]:
wb = xw.Book('себестоимостьА_в1.xlsx')
recipies = wb.sheets['Рецептура']

In [3]:
recipies.range('C7:C10').value

['Хлеб Венгерский', 'Хлеб "Барбари"', 'Багет парижский', 'Чиабатта']

1.1 На листе "Рецептура" для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

In [4]:
consumption = recipies.range('G7:O10').options(np.array).value
unit_prices = recipies.range('G14:O14').options(np.array).value
cost = consumption @ unit_prices

1.2 Результаты рассчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

In [5]:
# recipies.range('T7:T10').value = cost.reshape(-1, 1)
recipies.range('T6').value = 'Себестоимость'
recipies.range('T7').options(transpose=True).value = cost

1.3 Максимально приблизить форматирование столбца, добавленного в 1.2 к оформлению всей области.

In [6]:
# cost = [round(i, 2) for i in cost]
recipies.range('T7:T10').options(transpose=True).value = cost
xw.Range('T7:T10').number_format = '0,00'
xw.Range('T7:T10').color = (255, 255, 167)
recipies.range('T4:T6').merge()
recipies.range('T6').value = 'Себестоимость'
xw.Range('T4:T6').color = (255, 192, 0)
xw.Range('T4:T6').autofit()

1.4 Заменить значения добавленные в 1.2 на формулы. 

In [7]:
xw.Range('U7').formula = '=SUMPRODUCT(G7:O7,$G$14:$O$14)' 

1.5 Выполнить 1.4 с помощью "протягиваемых" формул.

In [8]:
xw.Range('U7').formula = '=SUMPRODUCT(G7:O7,$G$14:$O$14)' 
from xlwings.constants import AutoFillType 
xw.Range('U7').api.AutoFill(xw.Range("U7:U10").api, 
                                AutoFillType.xlFillDefault)

True

1.6 Сдлеать так, чтобы решение 1.5 и 1.3 автоматически применилось ко всем аналогичным областям на листе "Рецептура"

In [9]:
import xlwings as xw
import numpy as np
import pandas as pd
from xlwings import constants 

In [10]:
def readBookAndSheets(Book: str, Sheets: str) -> xw.Sheet:
    global wb, recipies
    wb = xw.Book(Book)
    recipies = wb.sheets[Sheets]
    return recipies
readBookAndSheets("себестоимостьА_в1.xlsx", "Рецептура")

<Sheet [себестоимостьА_в1.xlsx]Рецептура>

In [11]:
def getStartPoz() -> list:
    global start_poz
    start_poz = []
    for i, v in enumerate(recipies.range("B:B").value):
        if v == "№":
            start_poz.append(i)
    return start_poz

getStartPoz()

[3, 19, 36, 57]

In [12]:
def getTablesInfo() -> list:
    global tables_info   
    tables_info = []
    for start in start_poz:
        r = recipies.range(start+4, 2)
        poz_count = 0
        column_in_table = 0
        for poz in r.expand().value:
            if poz[1] is not None:
                poz_count += 1
            column_in_table += 1
        column_in_table -= 3
        tables_info.append(((start, 2), (start + r.expand().rows.count + 3, 2 + recipies.range((start, 2)).merge_area.columns.count - 1), poz_count, column_in_table))
    return tables_info
getTablesInfo()

[((3, 2), (16, 19), 4, 7),
 ((19, 2), (33, 18), 3, 8),
 ((36, 2), (54, 28), 7, 12),
 ((57, 2), (75, 29), 9, 12)]

In [13]:
from xlwings.constants import AutoFillType

def getTableDataCoords() -> list[dict]:
    tables_coords = []
    for start_coords, end_coords, poz_count, column_in_table in tables_info:

        data_start_coords = start_coords[0] + 4, start_coords[1] + 5
        data_end_coords = data_start_coords[0] + poz_count - 1, end_coords[1] - 1

        data_line_1 = (start_coords[0] + 4, start_coords[1] + 5), (data_start_coords[0], end_coords[1] - 1)

        unit_prices_coords_start = data_start_coords[0] + column_in_table, data_start_coords[1]
        unit_prices_coords_end = data_start_coords[0] + column_in_table, data_end_coords[1]
        
        cost_start = start_coords[0] + 4, end_coords[1] + 1
        cost_end = start_coords[0] + 4 + poz_count - 1, end_coords[1] + 1

        cost_start_2  = cost_start[0], cost_end[1] + 1
        cost_end_2 = cost_end[0], cost_end[1] + 1

        cost_label_data = cost_start[0]-3, cost_start[1]
        cost_label_all = (cost_label_data), (cost_label_data[0]+2, cost_label_data[1])

        sources_start = data_start_coords[0]-2, data_start_coords[1]
        sources_end = data_start_coords[0]-2, data_end_coords[1]

        tables_coords_dict = {
            "data": (data_start_coords, data_end_coords),
            "data_line_1": data_line_1,
            "unit_prices": (unit_prices_coords_start, unit_prices_coords_end),
            "cost_label": cost_label_data,
            "cost_label_merge": cost_label_all,
            "cost": (cost_start, cost_end),
            "cost2": (cost_start_2, cost_end_2),
            "sources": (sources_start, sources_end),

            }
        tables_coords.append(tables_coords_dict)
    return tables_coords

def calculateCost(tables_coords):
    for coords in tables_coords:

        consumption = recipies.range(*coords['data']).options(np.array).value
        consumption = np.nan_to_num(consumption)

        unit_prices = recipies.range(*coords['unit_prices']).options(np.array).value
        unit_prices = np.nan_to_num(unit_prices)

        cost = consumption @ unit_prices

        range_cost_label = recipies.range(coords['cost_label'])

        range_cost_label.value = 'Себестоимость'
        range_cost_label.api.Font.ColorIndex  = 2
        range_cost_label.api.Font.Bold = True
        range_cost_label.api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter
        range_cost_label.api.VerticalAlignment = xw.constants.VAlign.xlVAlignCenter
        range_cost_label.autofit()

        range_cost_marge = recipies.range(*coords['cost_label_merge'])

        range_cost_marge.merge()
        range_cost_marge.color = (255, 192, 0)
        range_cost_marge.api.Borders.Weight = 2

        range_cost = recipies.range(*coords['cost'])

        range_cost.number_format = '0,00'
        range_cost.options(transpose=True).value = cost
        range_cost.color = (255, 255, 167)
        range_cost.api.Borders.Weight = 2

        range_cost_2 = recipies.range(*coords['cost2'][0])

        range_cost_2.formula = f'=SUMPRODUCT(\
    {recipies.range(*coords["data_line_1"]).get_address(row_absolute = False , column_absolute = False )}, \
    {recipies.range(*coords["unit_prices"]).address})'
        range_cost_2.api.AutoFill(recipies.range(*coords['cost2']).api, AutoFillType.xlFillDefault)

calculateCost(tables_coords := getTableDataCoords())


## 2. Используя xlwings:

2.1 Собрать цены всех ресурсов, используемые на листе "Рецептура". Проверить, что нет расхождений между ценами совпадающих ресурсов

In [14]:
def getSourcesPrise(tables_coords: list[dict]) -> dict:
    sources_data = {}
    for coords in tables_coords:
        sources_name = recipies.range(*coords['sources']).options(np.array).value
        unit_prices = recipies.range(*coords['unit_prices']).options(np.array).value
        data_dict = dict(zip(sources_name, unit_prices))
        for key in data_dict.keys():
            if key == 'nan':
                continue
            value = sources_data.get(key)
            if value is None:
                sources_data[key] = data_dict[key]
            else:
                if value == data_dict[key]:
                    continue
                else:
                    print(f'расхождениt между ценами {key} {value} != {data_dict[key]}')
    return sources_data

sources_prise = getSourcesPrise(tables_coords=tables_coords)

2.2 Вынести цены ресурсов в новую таблицу на новом листе "Цена ресурсов".

In [15]:
try:
    prise = wb.sheets.add(name="Цена ресурсов", after="Рецептура")
except ValueError:
    prise = wb.sheets["Цена ресурсов"]

prise.range(1,1).value = list(sources_prise.items())

2.3 Заменить фиксированные цены ресурсов на листе "Рецептура" на ссылки на лист "Цена ресурсов"

In [55]:
def int_to_chr(x:int)->str:
    d, m = divmod(x, 26)
    m = chr(65 + m)
    if d > 0:
        return int_to_chr(d-1) + m
    else:
        return m

def changePriseStrToLink(tables_coords: list[dict], sources_prise):
    for index, (key, value) in enumerate(sources_prise.items()):
        sources_prise[key] = [index+1, value]
    

    for coords in tables_coords:
        source_coord = coords['sources'][0]
        sources = recipies.range(*coords['sources']).value
        links = ["='Цена ресурсов'!B" + str(sources_prise[key][0]) if key is not None else 0 for key in sources]
        unit_prices = recipies.range(*coords['unit_prices'])
        unit_prices.value = links

        
changePriseStrToLink(tables_coords=tables_coords, sources_prise=sources_prise)

2.4 Сделать так, чтобы цены ресурсов на листе "Рецептура" подтягивались с листа "Цена ресурсов" по наименованию (подсказка: использовать ВПР, рекомендуется использовать именованные области)

In [56]:
def changePriseStrToFormula(tables_coords: list[dict]):

    for coords in tables_coords:
        source_coord = coords['sources'][0]
        unit_prices = recipies.range(*coords['unit_prices'])
        unit_price = recipies.range(*coords['unit_prices'][0])
        unit_price.value = f"=IFERROR(VLOOKUP({int_to_chr(source_coord[1]-1) + str(source_coord[0])},'Цена ресурсов'!$A$1:$B$32,2,FALSE),0)"
        
        unit_price.api.AutoFill(unit_prices.api, AutoFillType.xlFillDefault)

        
changePriseStrToFormula(tables_coords=tables_coords)

## 3.

3.1 Создать нормализованную (не ниже 3й нормальной формы) базу данных sqlite для хранения информации из себестоимостьА_в1.xlsx .

Используя xlwings:

3.2 Информацию из себестоимостьА_в1.xlsx внести в БД на sqlite.

3.3 По базе данных sqlite сгенерирвать книгу эксель, по структуре аналогичную себестоимостьА_в1.xlsx

3.4 При помощи макроса на xlwings сделать в Excel кнопку, по которой все изменения на листе вносятся в БД, а цвет ячеек, в которых были найдены изменения, меняслся на зеленый.

3.5 Реализовать UDF, которая позволяет по имени ресурса вернуть количество видов в продукции в которых используется данный ресурс

3.6 Реализовать UDF, которая позволяет по имени ресурса вернуть массив наименований продуктов, в которых используется данный ресурс

3.7 Реазливать UDF 3.5 и 3.6 которые для получения ответа использую не книгу Excel а БД на sqlite.